In [10]:
import requests
import os
from langchain.docstore.document import Document
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI 
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma

In [11]:
embedding_function = AzureOpenAIEmbeddings(
    azure_endpoint="https://2000081253-openai.openai.azure.com/",
    api_key="107c20f6b6774c80b98c6f6a828f6374",
    azure_deployment="Text-embedding",
    api_version="2023-03-15-preview"
    )

In [12]:
model = AzureChatOpenAI(
    api_key="107c20f6b6774c80b98c6f6a828f6374",
    azure_deployment="gpt-4",
    azure_endpoint="https://2000081253-openai.openai.azure.com/",
    api_version="2023-03-15-preview"
)

In [13]:
base_url = os.environ["SERVICENOW_BASEURL"]
user = os.environ["SERVICENOW_USERNAME"]
pwd = os.environ["SERVICENOW_PASSWORD"]
headers = {"Content-Type":"application/json","Accept":"application/json"}

def get_catalog_item_collection():
    url = f'{base_url}/now/table/sc_cat_item?sysparm_query=sc_catalogsLIKEe0d08b13c3330100c8b837659bba8fb4%5Eactive%3Dtrue'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        data = {'result':{'sys_id': "", 'sys_name': "",'short_description':"",'description':""}, 'Status': response.status_code, 'Headers': response.headers, 'Error Response':response.json()}
    data = response.json()
    return data


def get_catalog_item(sys_id):
    url = f'{base_url}/sn_sc/servicecatalog/items/{sys_id}'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
    data = response.json()
    return data

def get_table_values(reference):
    url = f'{base_url}/now/table/{reference}'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()
    data = response.json()
    return data

def get_ref_qualifier_script(class_name):
    url = f'{base_url}/1353990/script_includes/script_include/{class_name}'

    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()

    data = response.json()
    print(data)

In [14]:
def descriptionMaker(data):

    print(data)

    template = """{data}
    
    using this data provide a proper description for this catalog item.
    also it should contain variable(parent and child) and variable description.
    In the Json data, If variables have reference_values, mention all of it in variable description.
    In the Json data, If variables have choices, mention all of it in variable description.
    The variables description should mention the ui policy condtions and actions if exist.
    The variables description should mention the reference qualifier.
    It should not contain other then that.
    
    Format:
    Catalog Item Name: Name of the catalog item
    Description: detailed description about catalog item.
    Variables (Fields):
    Name of the variable: About the variable.
    """
    
    prompt_template = PromptTemplate(template=template, input_variables=["data"])
    chain = prompt_template | model
    response = chain.invoke({"data": data})
    return response

In [15]:
catalog_item_collection = get_catalog_item_collection()

In [16]:
def set_reference_value(variable):
    if(variable['type'] == 8 and variable['dynamic_value_field'] == ''):
        reference = variable['reference']
        table_content = get_table_values(reference=reference)
        # print(table_content)
        names_in_table = []
        for content in table_content['result']:
            if('u_name' in content):
                names_in_table.append(content['u_name'])
            elif('name' in content):
                names_in_table.append(content['name'])
        variable['Choices'] = list(set(names_in_table))
        print(variable)
        return variable
    else:
        return variable
    

def get_all_variables_List(catalog_item_variable):
    def extract_variables(variables):
        variable_List = []
        for variable_info in variables:
            if variable_info['type'] == 24:
                continue
            if 'children' in variable_info:
                variable_List.extend(extract_variables(variable_info['children']))
            else:
                variable_List.append(variable_info)
        return variable_List
    
    return extract_variables(catalog_item_variable)




def arrange_response(catalog_item_json):
    print(catalog_item_json)
    variables = catalog_item_json['result']['variables']
    new_variables = get_all_variables_List(variables)
    final_result = []
    for variable in new_variables:
        if variable['type'] == 11:
            continue
        else:
            final_result.append(set_reference_value(variable))
    catalog_item_json['result']['variables'] = final_result
    return catalog_item_json



In [17]:
def createDocuments():
    docs=[]
    for catalog_item in catalog_item_collection['result']:
        if(catalog_item['sys_class_name'] != 'sc_cat_item'):
            continue
        if(catalog_item['sys_id'] != '39dbfcc3c3489a1068d8b132b4013167'):# '50572ffac3405a1068d8b132b4013177'):
            continue
        print(catalog_item)
        metadata = {"sys_id":catalog_item['sys_id'], "sys_name":catalog_item['sys_name'], "short_description":catalog_item["short_description"]}
        catalog_item_json = arrange_response(get_catalog_item(catalog_item['sys_id']))
        description = descriptionMaker(catalog_item_json)
        docs.append(Document(page_content=description.content, metadata=metadata))
        print(description.content)
    return docs


In [18]:
# from langchain_pinecone import PineconeVectorStore

# docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

In [19]:
query = "I need to install python"

In [20]:
path = './catalog_item_db'
isdir = os.path.isdir(path)
if(not isdir):
    db2 = Chroma.from_documents(documents=createDocuments(), embedding=embedding_function, persist_directory=path)

{'entitlement_script': '', 'use_sc_layout': 'true', 'show_variable_help_on_load': 'false', 'no_order_now': 'false', 'sc_ic_version': '', 'delivery_time': '1970-01-03 00:00:00', 'sys_updated_on': '2024-09-01 20:19:47', 'type': 'item', 'published_ref': '', 'no_search': 'false', 'sys_updated_by': 'system', 'price': '0', 'sys_created_on': '2024-08-20 12:43:10', 'recurring_frequency': '', 'sys_name': 'Report IT Incidents', 'model': '', 'state': '', 'no_cart': 'false', 'sys_created_by': 'admin', 'group': '', 'hide_sp': 'false', 'order': '0', 'start_closed': 'false', 'image': '', 'no_quantity': 'false', 'workflow': '', 'delivery_plan': {'link': 'https://dev203505.service-now.com/api/now/table/sc_cat_item_delivery_plan/523da512c611228900811a37c97c2014', 'value': '523da512c611228900811a37c97c2014'}, 'active': 'true', 'checked_out': '', 'custom_cart': '', 'no_cart_v2': 'false', 'no_proceed_checkout': 'false', 'version': '62', 'ignore_price': 'true', 'sys_update_name': 'sc_cat_item_39dbfcc3c3489a

In [21]:
docs = db2.similarity_search_with_relevance_scores(query)
print(docs)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


[(Document(metadata={'short_description': 'Get support with STG issues that you are facing', 'sys_id': '39dbfcc3c3489a1068d8b132b4013167', 'sys_name': 'Report IT Incidents'}, page_content='**Catalog Item Name:** Report IT Incidents\n\n**Description:** This catalog item allows users to report IT incidents they are experiencing and seek assistance for specific issues. The submission form gathers essential details to ensure the incident is accurately logged and addressed promptly.\n\n**Variables (Fields):**\n\n1. **Reported For:** This field captures the user for whom the incident is being reported. \n   - **Type:** Requested For\n   - **Reference:** `sys_user`\n   - **Attributes:** `edge_encryption_enabled=true`\n   - **Mandatory:** Yes\n   - **Dynamic Value:** `javascript:gs.getUserID();`\n\n2. **Location:** This field captures the location of the user reporting the incident. \n   - **Type:** Reference\n   - **Reference:** `cmn_location`\n   - **Attributes:** `edge_encryption_enabled=tr

c:\Users\2000081253\Desktop\Work\AI-Copilot-Ticket-Assistant\.venv\Lib\site-packages\langchain_core\vectorstores\base.py:784: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'short_description': 'Get support with STG issues that you are facing', 'sys_id': '39dbfcc3c3489a1068d8b132b4013167', 'sys_name': 'Report IT Incidents'}, page_content='**Catalog Item Name:** Report IT Incidents\n\n**Description:** This catalog item allows users to report IT incidents they are experiencing and seek assistance for specific issues. The submission form gathers essential details to ensure the incident is accurately logged and addressed promptly.\n\n**Variables (Fields):**\n\n1. **Reported For:** This field captures the user for whom the incident is being reported. \n   - **Type:** Requested For\n   - **Reference:** `sys_user`\n   - **Attributes:** `edge_encryption_enabled=true`\n   - **Mandatory:** Yes\n   - **Dynamic Value:** `javascript:gs.getUserID();`\n\n2. **Location

In [22]:
print(docs[0][0].page_content)

**Catalog Item Name:** Report IT Incidents

**Description:** This catalog item allows users to report IT incidents they are experiencing and seek assistance for specific issues. The submission form gathers essential details to ensure the incident is accurately logged and addressed promptly.

**Variables (Fields):**

1. **Reported For:** This field captures the user for whom the incident is being reported. 
   - **Type:** Requested For
   - **Reference:** `sys_user`
   - **Attributes:** `edge_encryption_enabled=true`
   - **Mandatory:** Yes
   - **Dynamic Value:** `javascript:gs.getUserID();`

2. **Location:** This field captures the location of the user reporting the incident. 
   - **Type:** Reference
   - **Reference:** `cmn_location`
   - **Attributes:** `edge_encryption_enabled=true`
   - **Dynamic Value Field:** `reported_for`
   - **Dynamic Value Dot Walk Path:** `location`
   - **Mandatory:** Yes

3. **Contact Number:** This field captures the contact number of the user reportin

In [23]:
docs

[(Document(metadata={'short_description': 'Get support with STG issues that you are facing', 'sys_id': '39dbfcc3c3489a1068d8b132b4013167', 'sys_name': 'Report IT Incidents'}, page_content='**Catalog Item Name:** Report IT Incidents\n\n**Description:** This catalog item allows users to report IT incidents they are experiencing and seek assistance for specific issues. The submission form gathers essential details to ensure the incident is accurately logged and addressed promptly.\n\n**Variables (Fields):**\n\n1. **Reported For:** This field captures the user for whom the incident is being reported. \n   - **Type:** Requested For\n   - **Reference:** `sys_user`\n   - **Attributes:** `edge_encryption_enabled=true`\n   - **Mandatory:** Yes\n   - **Dynamic Value:** `javascript:gs.getUserID();`\n\n2. **Location:** This field captures the location of the user reporting the incident. \n   - **Type:** Reference\n   - **Reference:** `cmn_location`\n   - **Attributes:** `edge_encryption_enabled=tr

In [24]:
# load from disk
# db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
# docs = db3.similarity_search(query)
# print(docs[0].page_content)

In [25]:
docs

[(Document(metadata={'short_description': 'Get support with STG issues that you are facing', 'sys_id': '39dbfcc3c3489a1068d8b132b4013167', 'sys_name': 'Report IT Incidents'}, page_content='**Catalog Item Name:** Report IT Incidents\n\n**Description:** This catalog item allows users to report IT incidents they are experiencing and seek assistance for specific issues. The submission form gathers essential details to ensure the incident is accurately logged and addressed promptly.\n\n**Variables (Fields):**\n\n1. **Reported For:** This field captures the user for whom the incident is being reported. \n   - **Type:** Requested For\n   - **Reference:** `sys_user`\n   - **Attributes:** `edge_encryption_enabled=true`\n   - **Mandatory:** Yes\n   - **Dynamic Value:** `javascript:gs.getUserID();`\n\n2. **Location:** This field captures the location of the user reporting the incident. \n   - **Type:** Reference\n   - **Reference:** `cmn_location`\n   - **Attributes:** `edge_encryption_enabled=tr

In [26]:
def fetchDescribe(data):

    template = """{data}
    
    from the given data Return only the 100 words of description of the catalog item.
    Output should not contain the variables
    And no premable or explaination.

    """
    
    prompt_template = PromptTemplate(template=template, input_variables=["data"])
    chain = prompt_template | model | StrOutputParser()
    response = chain.invoke({"data": data})
    return response

In [27]:
def fetchVariables(data, variables_name):

    template = """{data}
    
    from the given data Return only the variables name as a key and variable description as a value in JSON.
    The description must describe contains choices and ui_policy too as a single output
    The variablename should be one of the name from the given List. {variables_name}
    And no premable or explaination.

    Example:
    variablename : variabledescription

    """
    
    prompt_template = PromptTemplate(template=template, input_variables=["data", "variables_name"])
    chain = prompt_template | model | JsonOutputParser()
    response = chain.invoke({"data": data, "variables_name": variables_name})
    return response

In [28]:
variables = fetchVariables(docs[0][0].page_content, variables_name)

NameError: name 'variables_name' is not defined

In [ ]:
variables

{'requested_for': "This is a mandatory field to specify the user for whom the request is being made. The default value is set to the current user (System Administrator). UI Policy: If the selected 'Business Service' is 'Software Installation', this field becomes mandatory.",
 'contact_number': 'This is a mandatory single-line text field for entering the contact number of the requester.',
 'Location': "This is an optional reference field to specify the location. The value is dynamically fetched based on the 'Requested For' field.",
 'project_name': "This is an optional reference field to specify the project name. The value is dynamically fetched based on the 'Requested For' field.",
 'business_service': "This is a mandatory reference field to select the business service. Choices: OS Upgrade, Software Installation, Software Uninstallation. UI Policy: If the selected value is 'Software Installation' or 'Software Uninstallation', additional fields become mandatory and visible.",
 'software

In [ ]:
CATALOGITEMDOCS = "Document_Store\\catalog_item_db"
dir_path = os.path.join(os.getcwd(), CATALOGITEMDOCS)
isdir = os.path.isdir(dir_path)

In [ ]:
dir_path

'c:\\Users\\2000081253\\Desktop\\Work\\AI-Copilot-Ticket-Assistant\\src\\Draft-Folder\\Document_Store\\catalog_item_db'

In [ ]:
import sqlite3
import os

db_path = "c:\\Users\\2000081253\\Desktop\\Work\\AI-Copilot-Ticket-Assistant\\src\\Document_Store\\catalog_item_db\\chroma.sqlite3"

row_id = 0
catalog_item_description = ""
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT * FROM embedding_metadata") 
rows = cursor.fetchall()
for row in rows:
    if(row[1] == 'sys_id' and row[2] == '50572ffac3405a1068d8b132b4013177'):    
        row_id = row[0]
for row in rows:
    if(row[0] == row_id and row[1] == 'chroma:document'):
        catalog_item_description = row[2]
conn.close()
print(catalog_item_description)


Catalog Item Name: Software Services

Description: This catalog item allows users to request various software services, including OS upgrades, software installation, and uninstallation. Please select the appropriate business function/request type to display the delivery time. This service is available on desktop, and you can add multiple software items if required.

Variables (Fields):

1. Requested For: This is a mandatory field that specifies the user for whom the software service is requested. The default value is set to "System Administrator."

2. Contact Number: This is a mandatory field where you need to enter your contact number to facilitate communication.

3. Location: This field is not mandatory and allows you to specify the location related to the request. The location will be dynamically fetched based on the "Requested For" field.

4. Project Name: This field is not mandatory and allows you to specify the project associated with the request. The project name will be dynamic

In [ ]:
print(rows)

[(1, 'sys_id', '0317ba9d47120510f53d37d2846d43bb', None, None, None), (1, 'sys_name', 'Request email alias', None, None, None), (1, 'short_description', 'Request for email alias', None, None, None), (1, 'chroma:document', 'Catalog Item Name: Request email alias\n\nDescription: This catalog item allows users to request a new email alias. An email alias is an alternative email address that can be used to receive emails in addition to the primary email address. This can be particularly useful for managing different roles or functions without the need for multiple email accounts. To proceed with the request, users need to provide their preferred email alias and a business justification for why the alias is needed. \n\nVariables (Fields):\n1. Preferred email alias: \n   - About the variable: This field is mandatory and requires the user to enter the preferred email alias. The alias should be unique and follow the organization’s naming conventions. Edge encryption is enabled for this field t

In [ ]:
missing_variables = [{'active': True, 'label': 'Contact Number', 'dynamic_value_field': '', 'type': 6, 'mandatory': True, 'displayvalue': '', 'friendly_type': 'single_line_text', 'display_type': 'Single Line Text', 'render_label': True, 'read_only': False, 'name': 'contact_number', 'attributes': 'edge_encryption_enabled=true', 'id': 'cfa82f7ec3405a1068d8b132b4013157', 'value': '', 'dynamic_value_dot_walk_path': '', 'help_text': '', 'max_length': 0, 'order': 2}, {'ref_qual_elements': 'requested_for', 'lookup_label': '', 'active': True, 'label': 'Select your Host Name', 'dynamic_value_field': '', 'type': 18, 'mandatory': True, 'displayvalue': '', 'friendly_type': 'lookup_select_box', 'display_type': 'Lookup Select Box', 'render_label': True, 'lookup_table': 'u_host_name', 'read_only': False, 'lookup_value': 'u_name', 'name': 'select_your_host_name', 'attributes': 'edge_encryption_enabled=true,ref_qual_elements=requested_for', 'id': 'ca5133b2c3045a1068d8b132b4013185', 'choices': [{'recurring_price_currency': '', 'price': 0.0, 'index': 0, 'recurring_price': 0.0, 'label': 'HostName1', 'value': 'HostName1', 'price_currency': ''}, {'recurring_price_currency': '', 'price': 0.0, 'index': 1, 'recurring_price': 0.0, 'label': 'HostName2', 'value': 'HostName2', 'price_currency': ''}, {'recurring_price_currency': '', 'price': 0.0, 'index': 2, 'recurring_price': 0.0, 'label': 'HostName3', 'value': 'HostName3', 'price_currency': ''}], 'value': '', 'dynamic_value_dot_walk_path': '', 'help_text': '', 'max_length': 0, 'order': 28, 'reference_values': 'HostName1, HostName2, HostName3'}, {'name': 'duration', 'mandatory': 'true'}]

In [ ]:
fetched_variables = {'business_service': 'Software Installation',
  'software_name': 'Python',
  'add_software': 'true',
  'software_name1': 'Java',
  'add_software1': 'true',
  'software_name2': '.NET SDK',
  'requested_for': '6816f79cc0a8016401c5a33be04be441'}

missed_variables = ["duration", "select_your_host_name", "duration1", "duration2"]

In [ ]:
def prepare_missed_param_questions(catalog_description, fetched_variables, missed_variables):

    template = """{catalog_description}
    
    I have a catalog item with the following details:

    {fetched_variables}
    
    Some variables are missing from the request:

    {missed_variables}

    Please generate a JSON object with questions for each missing variable based on the given catalog item description and fetched data. 
    The JSON should include the following missing variables: {missed_variables}. 
    Do not include choices in the questions.

    Example:
    missedvariable1: question1 etc 

    And no premable or explaination.
            

    """
    prompt_template = PromptTemplate(template=template, input_variables=["catalog_description", "fetched_variables", "missed_variables"])
    chain = prompt_template | model | JsonOutputParser()
    response = chain.invoke({"catalog_description": catalog_description, "fetched_variables": fetched_variables, "missed_variables": missed_variables})
    return response

In [ ]:
value = prepare_missed_param_questions(catalog_item_description, fetched_variables, missed_variables)

In [ ]:
allquestions = value
questions = [ allquestions[i] for i in allquestions]

In [ ]:
questions

['Please specify the duration for which the software service is required for Python.',
 'Please select your host name.',
 'Please specify the duration for which the software service is required for Java.',
 'Please specify the duration for which the software service is required for .NET SDK.']